<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Deep History</b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>


<p style = 'font-size:16px;font-family:Arial'>Increasingly stringent regulations require companies to keep data online and accessible for regulatory compliance over many years. Although the most frequently accessed data is the latest or most current data, that doesn’t mean that the older information is not useful or relevant. Historical data that’s been compiled over the years gives a rich perspective of the business, such as long-term trends and cyclical patterns.</p>

<p style = 'font-size:16px;font-family:Arial'>Teradata Vantage provides unmatched concurrency and performance for the world's largest and most demanding enterprises to analyze their data. Analysis and concurrency needs for older information is generally substantially less as data ages, and over time there is much more history data that accumulates than current 'hot' data, it makes sense to store it somewhere that has different performance and price characteristics: for example an object store such as Amazon S3 or Azure Blob storage.</p>

<p style = 'font-size:16px;font-family:Arial'>Keeping historical and current data in separate systems can make it a challenge to gain unique insights that are possible only by analyzing the information together. But not any longer. Now, Teradata Vantage can be used to seamlessly join together all the historical and current information across the data warehouse AND object storage, without having to change the basic data structures and queries. This makes it possible to cost-effectively answer questions that could not be previously addressed so decision makers can better plan for the future.  When running this demo in the ClearScape Analytics Experience, the Vantage Database running on Google Cloud is accessing historical data that is stored on AWS S3. </p>

<img src= "images/aws_gcp_DeepHistory.png">

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>1. Start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [ ]:
Set query_band='DEMO=DeepHistory.ipynb;' update for session;

<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string. 


In [ ]:
call get_data('DEMO_SalesOffload_cloud'); ---takes about 35 seconds
---call get_data('DEMO_SalesOffload_local'); ---takes about 60 seconds

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
call space_report();  -- optional, takes about 10 seconds

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Querying the Data</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here is our current sales data. Let's grab some sample rows, we can see in this example we have customer, store, basket and discount information.</p>

In [ ]:
SELECT TOP 10 * 
FROM DEMO_SalesOffload.sales_fact;

<p style = 'font-size:16px;font-family:Arial'>Aggregating our current sales data for each date.</p>

In [ ]:
SELECT sales_date, sum(sales_quantity) as total 
FROM DEMO_SalesOffload.sales_fact
GROUP BY sales_date
ORDER BY sales_date ASC

<p style = 'font-size:16px;font-family:Arial'>Let's view our current sales data for each date by plotting it in a chart.</p>

In [ ]:
%chart sales_date, total, title='Recent Sales Trends', width=900, height=400

<p style = 'font-size:16px;font-family:Arial'>From the above chart it can be seen that the sales in the month of July, August and September are high. This is a sample data used to showcase this usecase and is not actual sales data.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Explore the offloaded historical data </b></p>

<p style = 'font-size:16px;font-family:Arial'>As you have seen we only have 1 year of sales data in our data warehouse as this is by far the most queried, but for compliance many companies need to keep up to 10 years of historical data. The older data has been exported from Vantage on a monthly basis and loaded into Amazon S3 for long term storage. With Teradata Vantage we can seamlessly access this offloaded data and join with the rest of the data to get insights over long term trends and handle audit requests with ease. This includes using existing queries and reports that would otherwise need to be re-written!</p>

<p style = 'font-size:16px;font-family:Arial'>We know the bucket where the offloaded sales data is located, so let's take a look at some of the data that is there - using the READ_NOS function we can get the list of files and their sizes.</p>

In [ ]:
SELECT location(char(255)), ObjectLength 
FROM read_nos (
USING 
 LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/') ---('/s3/s3.amazonaws.com/trial-datasets/SalesOffload')
 RETURNTYPE ('NOSREAD_KEYS')
) as d 
ORDER BY 1

<p style = 'font-size:16px;font-family:Arial'>There are total 114 files having sales data stored in the Object Store.</p>

<p style = 'font-size:16px;font-family:Arial'>Let's take a look at one of the files to get a better understanding of the file format:</p>

In [ ]:
SELECT * FROM READ_NOS (
      USING
      LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet')
      RETURNTYPE ('NOSREAD_PARQUET_SCHEMA')
      FULLSCAN ('TRUE')
      )
AS d

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Create a simple abstraction layer for easy access</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a foreign table and a view in Vantage to allow business analysts and other users to easily access the offloaded historical data:</p>

<p style = 'font-size:16px;font-family:Arial'>Note, you will need to list all the columns in the foreign table definition </p>

In [ ]:
CREATE FOREIGN TABLE sales_fact_offload
       (
Location VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC,
TheYear INTEGER,
TheMonth INTEGER,
sales_date DATE FORMAT 'YY/MM/DD',
customer_id INTEGER,
store_id INTEGER,
basket_id INTEGER,
product_id INTEGER,
sales_quantity INTEGER,
discount_amount FLOAT FORMAT '-ZZZ9.99'
)
USING
       (
LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/')
STOREDAS  ('PARQUET')
       )
NO PRIMARY INDEX
PARTITION BY COLUMN

<p style = 'font-size:16px;font-family:Arial'>Let's take a look at some of the rows that are in the offloaded files. The table is with the location of the file and than the columns as defined in the table created above.</p>

In [ ]:
SELECT TOP 10 *
FROM sales_fact_offload

<p style = 'font-size:16px;font-family:Arial'>How much data do we have out there? </p>

In [ ]:
SELECT COUNT(*)
FROM sales_fact_offload

<p style = 'font-size:16px;font-family:Arial'>Ok, we are close! We want the data to look like a native table. So let's put a view on top to split it out into columns.</p>

In [ ]:
REPLACE VIEW sales_fact_offload_v as (  
SELECT 
    sales_date,
    customer_id,
    store_id,
    basket_id,
    product_id,
    sales_quantity,
    discount_amount
FROM sales_fact_offload)

<p style = 'font-size:16px;font-family:Arial'>Now we can see that after creating the view with only the required columns we are able to get the required data , but the data is pulled at query runtime directly from the object store. We now have a seamless analytic experience by supporting the correlation of object store-based data sets with structured data sets in Teradata relational tables using existing SQL skills and workflows. </p>

In [ ]:
SELECT TOP 10 *
FROM sales_fact_offload_v

<p style = 'font-size:16px;font-family:Arial'>That looks nice! Now our users can access all the historical data we have in the object store.</p>

<p style = 'font-size:16px;font-family:Arial'>You can do everything in a view over a foreign table that you would do with a standard database view. This includes returning only a subset of the underlying table columns, as well as adding a WHERE clause in the view to limit what rows are made available using the view.</p>

<p style = 'font-size:16px;font-family:Arial'>Often we want to be able to look at just a portion of this vast amount of data at a time, which is why we have stored it by year and month. Let's re-define the foreign table to allow us to pre-filter the data when reading it.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Optimize the foreign table and view for efficient access</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have a lot of data in S3! Let's optimize the foreign table so that we minimize the data we have to read when querying the object store. Designing an object store bucket and path structure is an important first step when creating an object store. It requires knowledge of the business needs, the expected patterns in accessing the data, an understanding of the data, and a sensitivity to the trade-offs. In our case we will often know the approximate date we are looking at, so will use this to our advantage.</p>

In [ ]:
DROP TABLE sales_fact_offload

In [ ]:
CREATE FOREIGN TABLE sales_fact_offload
       (
Location VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC,
TheYear INTEGER,
TheMonth INTEGER,
sales_date DATE FORMAT 'YY/MM/DD',
customer_id INTEGER,
store_id INTEGER,
basket_id INTEGER,
product_id INTEGER,
sales_quantity INTEGER,
discount_amount FLOAT FORMAT '-ZZZ9.99'
)
USING
       (
LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/')
PATHPATTERN ('$dir1/$year/$month')
STOREDAS  ('PARQUET')
       )
NO PRIMARY INDEX
PARTITION BY COLUMN

<p style = 'font-size:16px;font-family:Arial'>We have re-defined our foreign table to include a <b>PATHPATTERN</b> clause. When looking at historical data by date, this allows us to read only the files we need!</p>

<p style = 'font-size:16px;font-family:Arial'>Now let's re-create our user-friendly view that allows for this path filtering...</p>

In [ ]:
REPLACE VIEW sales_fact_offload_v as (  
SELECT 
    CAST($path.$year AS CHAR(4)) sales_year,
    CAST($path.$month AS CHAR(2)) sales_month,
    sales_date,
    customer_id,
    store_id,
    basket_id,
    product_id,
    sales_quantity,
    discount_amount
FROM sales_fact_offload)

In [ ]:
SELECT TOP 10 *
FROM sales_fact_offload_v
WHERE sales_year = '2010'


<p style = 'font-size:16px;font-family:Arial'>This is great for use cases where we know the date. Suppose we need to see what a customer bought many years ago. Or maybe we want to report on historical store sales. The business analyst can easily query this with no IT intervention, no going to backups or other hard to reach data silos!</p>

<p style = 'font-size:16px;font-family:Arial'>Let's take a look at what store 6 did for sales back in 2012:</p>

In [ ]:
SELECT store_id, SUM(sales_quantity)
FROM sales_fact_offload_v
WHERE store_id = 6
AND sales_year = '2012'
GROUP BY 1

<p style = 'font-size:16px;font-family:Arial'>Let's join the historical data with the current data so we can see the full picture:</p>

In [ ]:
REPLACE VIEW sales_fact_all as (
SELECT 
    cast(Extract(year from Sales_Date) as CHAR(4)) sales_year,
    cast(Extract(month from Sales_date) as CHAR(2)) sales_month,
    sales_date,
    customer_id,
    store_id,
    basket_id,
    product_id,
    sales_quantity,
    discount_amount
    FROM DEMO_SalesOffload.sales_fact
    UNION ALL
SELECT
    sales_year,
    sales_month,
    sales_date,
    customer_id,
    store_id,
    basket_id,
    product_id,
    sales_quantity,
    discount_amount
FROM sales_fact_offload_v)

<p style = 'font-size:16px;font-family:Arial'>Final thing we will do is re-run our sales over time report, code is unchanged from the one above, we are now able to analyse all the sales data and not just the most recent year. Below we are trying to check sales for Store "8" over time.</p>

In [ ]:
SELECT sales_date, sum(sales_quantity) as total 
FROM sales_fact_all
where store_id = 8
GROUP BY sales_date
ORDER BY sales_date ASC

In [ ]:
%chart sales_date, total, title='All Sales Trends', width=900, height=400, color=green

<p style = 'font-size:16px;font-family:Arial'>Now we see that sales in 2019 were less as compared to 2017 and 2018. We will need to do further digging to see what happened. But thanks to Teradata Vantage we can cost-effectively analyse all our data by offloading the colder less required data to object storage for safe keeping.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Cleanup </b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
DROP VIEW sales_fact_all

In [ ]:
DROP VIEW sales_fact_offload_v

In [ ]:
DROP TABLE sales_fact_offload

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_SalesOffload');-- takes about 10 seconds, optional if you want to use the data later

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Dataset</b></p>

<p style = 'font-size:16px;font-family:Arial'>
The <b>sales_fact</b> dataset has approximately 43 million rows of sample sales data:

- `sales_date`: date the order was processed
- `customer_id`: customer identifier
- `store_id`: store identifier where the order was taken
- `basket_id`: grouping or order number
- `product_id`: identifier of the product
- `sales_quantity`: quantity of the product sold
- `discount_amount`: how much of a discount was given on this line item </p>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>